# Productionising the ML model
This notebook will walk through the process of creating, building and commiting the artifacts required to run the model developed in the Experimentation Notebook in production. 

## Environment Setup
**NOTE:** Set Project ID to your project  

In [1]:
PROJECT_ID = 'demokfp'
PREFIX = PROJECT_ID
REGION = 'us-central1'

DATA_ROOT = 'gs://workshop-datasets/covertype'
TRAINING_FILE_PATH = DATA_ROOT + '/training/dataset.csv'
VALIDATION_FILE_PATH = DATA_ROOT + '/evaluation/dataset.csv'

# Job dir for AI Platform Training
JOB_DIR_ROOT='gs://{}-artifact-store/jobs'.format(PREFIX)


NAMESPACE='kubeflow'
ZONE='us-central1-a'
ARTIFACT_STORE_URI='gs://{}-artifact-store'.format(PREFIX)
GCS_STAGING_PATH='{}/staging'.format(ARTIFACT_STORE_URI)
GKE_CLUSTER_NAME='{}-cluster'.format(PREFIX)

!gcloud container clusters get-credentials $GKE_CLUSTER_NAME --zone $ZONE
HOST_TEMP=!(kubectl describe configmap inverse-proxy-config -n $NAMESPACE | grep "googleusercontent.com")
INVERSE_PROXY_HOSTNAME=HOST_TEMP[0]


Fetching cluster endpoint and auth data.
kubeconfig entry generated for demokfp-cluster.


## Imports

In [2]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


## Import data set to BQ
Import the data set from cloud storage to BigQuery. A dataset is created and the table is imported under `covertype_data.covertype`

In [3]:
DATASET_LOCATION='US'
DATASET_ID='covertype_dataset'
TABLE_ID='covertype'
DATA_SOURCE='gs://workshop-datasets/covertype/full/dataset.csv'
SCHEMA='Elevation:INTEGER,\
Aspect:INTEGER,\
Slope:INTEGER,\
Horizontal_Distance_To_Hydrology:INTEGER,\
Vertical_Distance_To_Hydrology:INTEGER,\
Horizontal_Distance_To_Roadways:INTEGER,\
Hillshade_9am:INTEGER,\
Hillshade_Noon:INTEGER,\
Hillshade_3pm:INTEGER,\
Horizontal_Distance_To_Fire_Points:INTEGER,\
Wilderness_Area:STRING,\
Soil_Type:STRING,\
Cover_Type:INTEGER'

!bq --location=$DATASET_LOCATION --project_id=$PROJECT_ID mk --dataset $DATASET_ID
!bq --project_id=$PROJECT_ID --dataset_id=$DATASET_ID load \
--source_format=CSV \
--skip_leading_rows=1 \
--replace \
$TABLE_ID \
$DATA_SOURCE \
$SCHEMA

BigQuery error in mk operation: Dataset 'demokfp:covertype_dataset' already
exists.
Waiting on bqjob_re662610dab0ecf4_000001703771d97c_1 ... (9s) Current status: DONE   


## Prepare the training application.
Now that the data is hosted in BQ, the next step is to create the training application. Start by creating the folders to host the model script, trainer image docker and the base image docker.

In [4]:
TRAINING_APP_FOLDER = 'trainer_image'
BASE_IMAGE_FOLDER='base_image'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
os.makedirs(BASE_IMAGE_FOLDER, exist_ok=True)

### Write the training script. 

The script written in the Experimentation Notebook which process the data and trains the classification model is written as a training script `train.py` in the training image folder. In addition to the model written during experimentation an additional `hypertune` function is created which allows for a training job to be run with multiple parameters. This will  run multiple models with a range of parameter. 

In [5]:
%%writefile {TRAINING_APP_FOLDER}/train.py
"""Covertype Classifier trainer script."""

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
  df_train = pd.read_csv(training_dataset_path)
  df_validation = pd.read_csv(validation_dataset_path)
    
  if not hptune:
    df_train = pd.concat([df_train, df_validation])

  numeric_feature_indexes = slice(0, 10)
  categorical_feature_indexes = slice(10, 12)

  preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

  pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log'))
  ])
    
  num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
  df_train = df_train.astype(num_features_type_map)
  df_validation = df_validation.astype(num_features_type_map) 

  print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
  X_train = df_train.drop('Cover_Type', axis=1)
  y_train = df_train['Cover_Type']
  
  pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
  pipeline.fit(X_train, y_train)
  
  if hptune:
    X_validation = df_validation.drop('Cover_Type', axis=1)
    y_validation = df_validation['Cover_Type']
    accuracy = pipeline.score(X_validation, y_validation)
    print('Model accuracy: {}'.format(accuracy))
    # Log it with hypertune
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=accuracy
    )

  # Save the model
  if not hptune:
    model_filename = 'model.pkl'
    with open(model_filename, 'wb') as model_file:
        pickle.dump(pipeline, model_file)
    gcs_model_path = "{}/{}".format(job_dir, model_filename)
    subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
    print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  fire.Fire(train_evaluate)

Overwriting trainer_image/train.py


### Package the script into a docker image.

The docker images used for the training are based off the image `mlops-dev:TF115-TFX015-KFP136` created during the inital set up of the environment. Since the AI Platform Notebook instance is based on the `mlops-dev:TF115-TFX015-KFP136` image we use the same image as a base for the training image. 


We first write a base image dockerfile which replicates the image used for the Notebook. Then we write a training dockerfile which uses the same base image and add the `train.py` to the image. 


**NOTE:** Make sure to update the URI for the image so that it points to your project's **Container Registry**. i.e. `FROM gcr.io/PROJECT_ID/mlops-dev:TF115-TFX015-KFP136` 

In [6]:
%%writefile {BASE_IMAGE_FOLDER}/Dockerfile
FROM gcr.io/demokfp/mlops-dev:TF115-TFX015-KFP136

Overwriting base_image/Dockerfile


In [7]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/demokfp/mlops-dev:TF115-TFX015-KFP136
RUN pip install -U fire cloudml-hypertune
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting trainer_image/Dockerfile


## Build base and trainer images 
Use **Cloud Build** to build the images and save them to the **Cloud Container Registery**.

In [8]:
IMAGE_URI_BASE="gcr.io/{}/{}:latest".format(PROJECT_ID,BASE_IMAGE_FOLDER)

!gcloud builds submit --timeout 15m --tag {IMAGE_URI_BASE} base_image

Creating temporary tarball archive of 1 file(s) totalling 49 bytes before compression.
Uploading tarball of [base_image] to [gs://demokfp_cloudbuild/source/1581478226.9-cc40f832117d42449c6931573d4b1789.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/demokfp/builds/43d92e1d-b6af-41fd-9156-6917e2ef850f].
Logs are available at [https://console.cloud.google.com/gcr/builds/43d92e1d-b6af-41fd-9156-6917e2ef850f?project=435903989237].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "43d92e1d-b6af-41fd-9156-6917e2ef850f"

FETCHSOURCE
Fetching storage object: gs://demokfp_cloudbuild/source/1581478226.9-cc40f832117d42449c6931573d4b1789.tgz#1581478227104329
Copying gs://demokfp_cloudbuild/source/1581478226.9-cc40f832117d42449c6931573d4b1789.tgz#1581478227104329...
/ [1 files][  176.0 B/  176.0 B]                                                
Operation completed over 1 objects/176.0 B.                                      
BUILD
Already 

In [9]:
IMAGE_URI_TRAIN="gcr.io/{}/{}:latest".format(PROJECT_ID,TRAINING_APP_FOLDER)

!gcloud builds submit --timeout 15m --tag {IMAGE_URI_TRAIN} trainer_image

Creating temporary tarball archive of 3 file(s) totalling 2.6 KiB before compression.
Uploading tarball of [trainer_image] to [gs://demokfp_cloudbuild/source/1581478459.63-166b3a51b65245e281f7907f22e813a6.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/demokfp/builds/e9827cdb-331b-4260-a291-379dd4a05f6d].
Logs are available at [https://console.cloud.google.com/gcr/builds/e9827cdb-331b-4260-a291-379dd4a05f6d?project=435903989237].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e9827cdb-331b-4260-a291-379dd4a05f6d"

FETCHSOURCE
Fetching storage object: gs://demokfp_cloudbuild/source/1581478459.63-166b3a51b65245e281f7907f22e813a6.tgz#1581478459897769
Copying gs://demokfp_cloudbuild/source/1581478459.63-166b3a51b65245e281f7907f22e813a6.tgz#1581478459897769...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 objects/1.2 KiB.                                      
BUILD
Alr

## Create Yaml file of the pipeline
Now we build the the yaml file for the pipeline based on a file `covertype_training_pipeline.py`. This script outlines the pipeline and the individual components and the appropriate input and outputs. Using the DSL compile command, a static configuration (in YAML format) is created that the Kubeflow Pipelines can execute. 

**NOTE:** Change the environment settings in `covertype_training_pipeline.py` to reflect your BASE_IMAGE and TRAINER_IMAGE URI's. i.e *'gcr.io/PROJECT_ID/base_image:latest'*

In [10]:
!dsl-compile --py covertype_training_pipeline.py --output covertype_training_pipeline.yaml

## Deploying the pipeline
Select a pipeline name, ensure it is not already in use (else a 500 error will be displayed). Deploy the pipeline. 

In [11]:
PIPELINE_NAME='covertype_classifier_training1'

!kfp --endpoint {INVERSE_PROXY_HOSTNAME} pipeline upload -p {PIPELINE_NAME} covertype_training_pipeline.yaml

Pipeline ab90cc6c-3f00-480c-bea3-bd1959e7394b has been submitted

Pipeline Details
------------------
ID           ab90cc6c-3f00-480c-bea3-bd1959e7394b
Name         covertype_classifier_training1
Description
Uploaded at  2020-02-12T03:38:20+00:00
+-----------------------------+--------------------------------------------------+
| Parameter Name              | Default Value                                    |
+=============================+==================================================+
| project_id                  |                                                  |
+-----------------------------+--------------------------------------------------+
| region                      |                                                  |
+-----------------------------+--------------------------------------------------+
| source_table_name           |                                                  |
+-----------------------------+--------------------------------------------------+
| gcs_

This command will return a list of pipelines depolyed at the given hostname. We see that `covertype_classifier_training` has been deployed. This list also allows us to copy the pipeline ID. 

In [ ]:
!kfp --endpoint {INVERSE_PROXY_HOSTNAME} pipeline list

#### Viewing the pipeline
The deployed pipeline can be viewed through the Kubeflow Pipeline UI given at the URL below. 

In [12]:
print('https://{}'.format(INVERSE_PROXY_HOSTNAME))

https://21bce3d410fd3c82-dot-us-central1.notebooks.googleusercontent.com


## Run Experiment 
Now that the pipeline is deployed we want to run an experiment, this will cause the pipeline to run, pulling the data from bigquery and splitting it, training the models, evaluating them and deploy the best performing model. This experiment takes approximately an hour to execute and will result in a deployed model which can be interacted with through GCP's AI platform predicting service. 

**NOTE:** Change the PIPELINE_ID to reflect the ID copied from above.  

In [13]:
PIPELINE_ID='ab90cc6c-3f00-480c-bea3-bd1959e7394b'

EXPERIMENT_NAME='Covertype_Classifier_Training'
RUN_ID='Run_001'
SOURCE_TABLE='covertype_dataset.covertype'
DATASET_ID='splits'
EVALUATION_METRIC='accuracy'
EVALUATION_METRIC_THRESHOLD='0.69'
MODEL_ID='covertype_classifier'
VERSION_ID='v01'
REPLACE_EXISTING_VERSION=True

In [14]:
!kfp --endpoint {INVERSE_PROXY_HOSTNAME} run submit \
-e Covertype_Classifier_Training \
-r {RUN_ID} \
-p {PIPELINE_ID} \
project_id={PROJECT_ID} \
gcs_root={GCS_STAGING_PATH} \
region={REGION} \
source_table_name={SOURCE_TABLE} \
dataset_id={DATASET_ID} \
evaluation_metric_name={EVALUATION_METRIC} \
evaluation_metric_threshold={EVALUATION_METRIC_THRESHOLD} \
model_id={MODEL_ID} \
version_id={VERSION_ID} \
replace_existing_version={REPLACE_EXISTING_VERSION}

Run 8d0a4358-ea11-4dcf-af1c-582e4ca8f418 is submitted
+--------------------------------------+---------+----------+---------------------------+
| run id                               | name    | status   | created at                |
+======================================+=========+==========+===========================+
| 8d0a4358-ea11-4dcf-af1c-582e4ca8f418 | Run_001 |          | 2020-02-12T03:38:55+00:00 |
+--------------------------------------+---------+----------+---------------------------+
